# Adapted csv file for datasets

In [8]:
import os
import numpy as np
import cv2
import pandas as pd 
import csv

## read data

In [9]:
root_path = os.getcwd() 
images_path = os.path.join(root_path,'images')
data_path = os.path.join(root_path, 'scores')

dataset_o =  os.path.join(data_path, 'score_overall.csv')
dataset_c =  os.path.join(data_path, 'score_colorfulness.csv')
dataset_ch=  os.path.join(data_path, 'score_color_harmony.csv')
dataset_f =  os.path.join(data_path, 'score_fineness.csv')

ids_o = pd.read_csv(dataset_o)
ids_c = pd.read_csv(dataset_c)
ids_ch= pd.read_csv(dataset_ch)
ids_f = pd.read_csv(dataset_f)

## Generate test list and valid list

In [10]:
game = os.listdir(images_path)
n = len(game)
# test = np.empty([n,1], dtype = str) 
# valid = np.empty([n,1], dtype = str)
test = []
valid =[]
for i in range(n):
    name_game = game[i]
    game_path = os.path.join(images_path,name_game)
    images = os.listdir(game_path)
    if 0<len(images)<3 :
        test_image = images[0]
        valid_image = ''
    else :
        img = np.random.choice(images, size=2, replace=False, p=None)
        test_image = img[0]
        valid_image = img[1]
    test.append(name_game + '/' + str(test_image))
    valid.append(name_game + '/' + str(valid_image))

## write data

In [11]:
with open(os.path.join(data_path, "all.csv"),"w") as csvfile:
    writer = csv.writer(csvfile)
    # write columns_name
    writer.writerow(["image_name", "MOS_overall", "MOS_fineness", "MOS_colorful", "MOS_harmony", "set", "width", "height"])

    # write parameters of each images
    for i in range(len(ids_o.index)):
        img_name = ids_o.loc[i, 'filename']
        
        mos_o = ids_o.loc[i, 'mean']
        mos_f = ids_f.loc[ids_f.filename == img_name, 'mean'].iloc[0]
        mos_c = ids_c.loc[ids_c.filename == img_name, 'mean'].iloc[0]
        mos_ch = ids_ch.loc[ids_ch.filename == img_name, 'mean'].iloc[0]
        
        index = img_name.rfind('_', 0, len(img_name)-7)
        img_name = img_name[:index] + '/' +img_name[index+1:]
        
        if img_name in test:
            sett = 'test'
        elif img_name in valid:
            sett = 'validation'
        else:
            sett = 'training'
        
        img_path = os.path.join(images_path, img_name)
        if not os.path.exists(img_path):
            print("\n Not Found :" + img_path)
            print("Number :" + str(i))
            continue
        img = cv2.imread(img_path)
        sp = img.shape
        width = sp[1]
        height = sp[0]
        writer.writerow([img_name,mos_o,mos_f,mos_c,mos_ch,sett,width,height])